In [12]:
import math
import collections
import itertools
from functools import cached_property
import warnings
from pathlib import Path
import awkward as ak
import pandas as pd
import numpy as np
import uproot
import vector

import plotly.express as px
import plotly.graph_objects as go
from hsluv import hex_to_hsluv, hsluv_to_hex


class LoadedEvent:
    def __init__(self, path, eventNb=1, positiveEndcap=True):
        self.positiveEndcap = positiveEndcap
        self.f = uproot.open(path)["ticlDumper"]
        self.eventNb = eventNb
        self.cmsswEventNb = self.f["trackstersCLUE3DHigh;1"].arrays(filter_name="event*")["event_"][self.eventNb]
        clus2D_ak = self.f["clusters"].arrays(library="ak")
        self.clus2D_df = ak.to_dataframe(clus2D_ak[[field for field in clus2D_ak.fields if field != "cluster_hits_indexes"]], levelname=lambda i : {0:"event", 1:'cluster'}[i])
        self.rechits_df_unjoined = ak.to_dataframe(self.f["rechits"].arrays(library="ak"), levelname=lambda i : {0:"event", 1:'rechit'}[i])

        assoc = ( # best association to CP for each trackster
            ak.to_dataframe(self.f["associations"].arrays(filter_name="tsCLUE3D_recoToSim_CP*"), levelname=lambda i : {0:"event", 1:'trackster', 2:"simts"}[i])
            .sort_values(["event", "trackster", "tsCLUE3D_recoToSim_CP_score"], ascending=True)
            .loc[:, :, 0]
        )

        self.ts_df = ak.to_dataframe(self.f["trackstersCLUE3DHigh"].arrays(library="ak", filter_name=["raw_energy", "barycenter_*"]), levelname=lambda i : {0:"event", 1:'trackster'}[i]).join(assoc).loc[eventNb]
        self.leading_trackster = self.ts_df[self.ts_df.barycenter_z * (1 if positiveEndcap else -1) > 0].sort_values("raw_energy", ascending=False).iloc[0]

        sc = self.f["superclustering"]["linkedResultTracksters"].array()[self.eventNb]
        self.leading_supercluster = sc[list(sc[:, 0]).index(self.leading_trackster.name)]


        ts_to_clus = (
            ak.to_dataframe(self.f["trackstersCLUE3DHigh"].arrays(library="ak", filter_name=["vertices_indexes", "raw_energy"]), levelname=lambda i : {0:"event", 1:'trackster', 2:"cluster_in_ts"}[i])
            .join(assoc) # add association score to CP
            .reset_index(["trackster", "cluster_in_ts"])
            .set_index("vertices_indexes", append=True)
        )
        
        
        clus_to_rechits = ak.to_dataframe(self.f["clusters"].arrays(library="ak", filter_name=["cluster_layer", "cluster_hits_indexes"]), levelname=lambda i : {0:"event", 1:'cluster', 2:"rechit_in_cluster"}[i]).reset_index("rechit_in_cluster")
        #clus_to_rechits = clus_to_rechits[clus_to_rechits.cluster_hits_indexes != 0] # 
        self.rechits_df_mul = (ts_to_clus
            .join(clus_to_rechits, on=["event", "vertices_indexes"], how="inner")
            .reset_index("vertices_indexes")
            .set_index("cluster_hits_indexes", append=True)
            .join(self.rechits_df_unjoined, on=["event", "cluster_hits_indexes"], how="right")
        )

        self.rechits_df = self.rechits_df_mul.loc[eventNb]
    

def NaNColorMap(d:dict[float, str], NaNColor:str):
    def mapFct(val:float) -> str:
        if math.isnan(val):
            return NaNColor
        else:
            return d[val]
    return mapFct


class MarkerSizeLinearScaler:
    def __init__(self, allEnergiesSeries:pd.Series, maxMarkerSize=10) -> None:
        self.maxEnergy = allEnergiesSeries.max()
        self.maxMarkerSize = maxMarkerSize
    
    def scale(self, series:pd.Series):
        return (series / self.maxEnergy * self.maxMarkerSize).clip(lower=1)

class MarkerSizeLogScaler:
    def __init__(self, allEnergiesSeries:pd.Series, maxMarkerSize=10, minMarkerSize=1) -> None:
        """ Log scale such that min(allEnergiesSeries) maps to minMarkerSize, and max(allEnergiesSeries) maps to maxMarkerSize
        Write size = b * ln(E/a) """
        minEnergy = allEnergiesSeries.min()
        maxEnergy = allEnergiesSeries.max()
        if minEnergy < maxEnergy:
            self._ln_a = (maxMarkerSize * math.log(minEnergy) - minMarkerSize*math.log(maxEnergy)) / (maxMarkerSize - minMarkerSize)
            self._b = minMarkerSize / (math.log(minEnergy) - self._ln_a)
        else:
            # Deal with the case with only one energy
            self._ln_a = math.log(minEnergy) - 1
            self._b = maxMarkerSize # Put desired marker size here. For now just take the max

    def scale(self, val:pd.Series): # can be pd.Series or float
        if isinstance(val, pd.Series):
            return (self._b * (np.log(val) - self._ln_a)).clip(lower=1)
        else:
            return max(1, self._b * (np.log(val) - self._ln_a))
    


def getPointTypeStringForRechits(clus2D_id:float, grouped_df:pd.DataFrame):
    """ 
    Parameters : 
    - clus2D_id : the 2D cluster id of the grouped_df (can be NaN in case not in a layer cluster or in a masked layer cluster)
    - grouped_df : rechitds_df (must have rechits_pointType)"""
    pointTypeString = []
    pointTypeDict = {0:"Follower", 1:"Seed", 2:"Outlier"}
    for row in grouped_df.itertuples():
        if row.rechits_pointType == 1 and math.isnan(clus2D_id): # Seed, but clus2D_id is NaN
            pointTypeString.append("Masked cluster seed")
        else:
            pointTypeString.append(pointTypeDict[row.rechits_pointType])
    return pointTypeString


class LayerVisualization:
    def __init__(self, event:LoadedEvent, layerNb:int, trackster_color_map=dict(), standalone=False, sizeSettings=dict(), center_x=None, center_y=None, default_color_cycle = itertools.cycle(px.colors.qualitative.Dark24)) -> None:
        """ 
        Parameters : 
         - standalone : if False, meant to be plotted with a 3D view of the event, with coeherent colors. If True, meant as standalone plot (choose best colors) 
        """
        self.event:LoadedEvent = event
        self.sizeSettings = sizeSettings
        self.layerNb = layerNb
        self.standalone = standalone
        self.fig = go.Figure(
            layout=go.Layout(
                title=go.layout.Title(text=f"Layer visualization - Layer {layerNb}"),
                #width=1200,
                #height=600,
                autosize=True
            )
        )
        self.fig.update_yaxes( # Same scale for x and y
            scaleanchor="x",
            scaleratio=1,
        )
        self.trackster_color_map = trackster_color_map
        self.default_color_cycle = default_color_cycle
        self.center_x, self.center_y = center_x, center_y


        
    @property
    def rechits_df_onLayer(self) -> pd.DataFrame:
        sel = (self.event.rechits_df.rechit_layer == self.layerNb) & (self.event.rechits_df.rechit_z * (1 if self.event.positiveEndcap else -1) > 0) 
        if self.center_x is not None:
            sel &= ((self.event.rechits_df.rechit_x-self.center_x)**2 + (self.event.rechits_df.rechit_y-self.center_y)**2) < 30**2
        return self.event.rechits_df[sel]

    @cached_property
    def totalEnergyOnLayer(self) -> float:
        return self.rechits_df_onLayer.rechits_energy.sum()
    @cached_property
    def maxRechitEnergyOnLayer(self) -> float:
        return self.rechits_df_onLayer.rechits_energy.max()
    
    def add2DClusters(self):
        """ Add symbols of each layer cluster """
        clus2DMarkerSizeScale = MarkerSizeLogScaler(self.event.clus2D_df.clus2D_energy, maxMarkerSize=self.sizeSettings.get("clue.max", 50), minMarkerSize=self.sizeSettings.get("clue.min", 15))
        outlier_counter = 1
        for clus2D in self.clus2D_df_onLayer.sort_values("clus2D_energy", ascending=False).itertuples():
            clus2D_id = clus2D.Index
            # The symbol depends on the associated trackster
            if math.isnan(clus2D.clus3D_id):
                # LC not in a trackster : a symbol per LC
                clus3D_id_symbol = next(self.clus3D_symbols_outlier_2Dview)
                legend_name = f"Nb {clus2D_id} - (not in a trackster)"
                outlier_counter += 1
            else:
                # LC in a trackster : use the trackster symbol
                clus3D_id_symbol = self.mapClus3Did_symbol_2Dview[clus2D.clus3D_id]
                legend_name = f"Nb {clus2D_id} - Trackster {int(clus2D.clus3D_id)}"
            
            self.fig.add_trace(go.Scatter(
                mode="markers",
                legendgroup="cluster2D",
                legendgrouptitle_text="2D clusters",
                name=legend_name,
                x=[clus2D.clus2D_x], y=[clus2D.clus2D_y],
                opacity=self.sizeSettings.get("clue.opacity", 0.8),
                marker=dict(
                    symbol=clus3D_id_symbol,
                    color=self.mapClus2Did_color(clus2D_id),
                    line_color="black",
                    line_width=2, # Does not work on some graphics cards
                    size=clus2DMarkerSizeScale.scale(clus2D.clus2D_energy),
                ),
                customdata=[[clus2D.clus2D_energy, clus2D.clus2D_rho, clus2D.clus2D_delta,
                    {0:"Follower", 1:"Seed", 2:"Outlier"}[clus2D.clus2D_pointType]]],
                #hovertemplate="clus2D_x=%{x}<br>clus2D_y=%{y}<br>clus2D_z=%{z}<br>clus2D_size=%{marker.size}<extra></extra>",
                hovertemplate=(
                    "2D cluster : %{customdata[3]}<br>"
                    "Energy: %{customdata[0]:.2g} GeV<br>Rho: %{customdata[1]:.2g} GeV<br>"
                    "Delta: %{customdata[2]:.2g} cm<br>"
                    "Size: %{marker.size}"
                )
            ))

        return self
    
    def addRechits(self, energyForScale=None):
        markerSizeScale = MarkerSizeLogScaler(self.rechits_df_onLayer.rechit_energy if energyForScale is None else energyForScale, maxMarkerSize=self.sizeSettings.get("rechits.max", 25), minMarkerSize=self.sizeSettings.get("rechits.min", 2))
        #markerSizeScale = MarkerSizeLinearScaler(self.rechits_df_onLayer.rechits_energy, maxMarkerSize=10)
        showLegend = True
        for index, grouped_df in self.rechits_df_onLayer.groupby(by=["trackster"], dropna=False):
            # dropna=False to keep outlier rechits and rechits members of outlier layer cluster
            trackster_nb = index[0]
            
            # split PU/nonPU
            noPU_df, PU_df = grouped_df[grouped_df.rechit_nonPUFraction > 0.5], grouped_df[grouped_df.rechit_nonPUFraction <= 0.5]
            isTracksterPUOnly = grouped_df.tsCLUE3D_recoToSim_CP_score.iloc[0] > 0.5

            try:
                ts_color = self.trackster_color_map[trackster_nb]
            except KeyError:
                ts_color = next(self.default_color_cycle)
            if math.isnan(trackster_nb): 
                ts_color = "#222A2A"

            
            def makeCustomData(df):
                pos = vector.arr(dict(x=df["rechit_x"], y=df["rechit_y"], z=df["rechit_z"]))
                return np.stack( (
                    df["rechit_energy"]*1000, 
                    df["tsCLUE3D_recoToSim_CP_score"],
                    df["tsCLUE3D_recoToSim_CP_sharedE"],
                    df["tsCLUE3D_recoToSim_CP_sharedE"]/df["raw_energy"]*100,
                    df["rechit_nonPUFraction"]*100,
                    pos.eta,
                    pos.phi
                    ), axis=-1)
            hovertemplate = (
                    "Rechit : %{customdata[0]:.2g} MeV - eta %{customdata[5]:.2f} - phi %{customdata[6]:.2f}<br>"
                    "Non-PU fraction : %{customdata[4]} %"
                    "Assoc score : %{customdata[1]:.2g}<br>"
                    "SharedE : %{customdata[2]} GeV<br>"
                    "SharedE/TsEnergy : %{customdata[3]:.2g} %"
                )
            if len(noPU_df) > 0:
                # noPU
                #print(makeCustomData(noPU_df))
                self.fig.add_trace(go.Scatter(
                    mode="markers",
                    legendgroup="rechits",
                    legendgrouptitle_text="Rechits",
                    name=f"Trackster nb {trackster_nb}",
                    x=noPU_df["rechit_x"], y=noPU_df["rechit_y"], 
                    marker=dict(
                        symbol="circle",
                        color=ts_color,
                        size=markerSizeScale.scale(noPU_df["rechit_energy"]),
                        opacity=1.,
                    ),
                    customdata=makeCustomData(noPU_df),
                    #hovertemplate="clus2D_x=%{x}<br>clus2D_y=%{y}<br>clus2D_z=%{z}<br>clus2D_size=%{marker.size}<extra></extra>",
                    hovertemplate=hovertemplate
                ))
            if len(PU_df) > 0:
                # PU
                self.fig.add_trace(go.Scatter(
                    mode="markers",
                    legendgroup="rechits",
                    legendgrouptitle_text="Rechits",
                    name=f"Trackster nb {index[0]}",
                    x=PU_df["rechit_x"], y=PU_df["rechit_y"], 
                    marker=dict(
                        symbol="circle",
                        color="rgba(0, 0, 0, 0)",
                        size=markerSizeScale.scale(PU_df["rechit_energy"]),
                        opacity=1.,
                        line=dict(
                            color=ts_color,
                            width=2,
                            
                        )
                    ),
                    opacity=0.5 if isTracksterPUOnly else 1,
                    customdata=makeCustomData(PU_df),
                    #hovertemplate="clus2D_x=%{x}<br>clus2D_y=%{y}<br>clus2D_z=%{z}<br>clus2D_size=%{marker.size}<extra></extra>",
                    hovertemplate=hovertemplate
                ))



        return self

    
    def addHexagonSensors(self, fill=True):
        # try:
        #     df_sensor_positions = pd.read_hdf(Path(__file__).resolve().parent / "cells-positions.hdf5", self.event.datatype)
        # except Exception as e:
        #     warnings.warn("Could not load sensors locations : " + str(e))
        #     return self
        
        #sensors_onLayer = df_sensor_positions.loc[self.layerNb]
        #for sensor in sensors_onLayer.itertuples():
        #sensors_layer = sensors[np.isclose(sensors[:, 2], self.rechits_df_onLayer.iloc[0].rechit_z, atol=0.5)]
        sensors_layer = sensors[sensors.rechit_layer == self.layerNb]
        if self.center_x is not None:
            sensors_layer = sensors_layer[((sensors_layer.rechit_x-self.center_x)**2 + (sensors_layer.rechit_y-self.center_y)**2) < 10**2]
        print(sensors_layer)
        for sensor in sensors_layer.itertuples():
            x = sensor.rechit_x
            y = sensor.rechit_y
            # low density ?
            #radius = (0.974487-9.009615e-06)/np.sqrt(3)*1.07
            # high density
            radius = (84.069237-83.48784 )/np.sqrt(3)*1.07
            
            vertices = np.array([
                [x+radius,y],
                [x+radius/2,y+(np.sqrt(3.)/2)*radius],
                [x-radius/2,y+(np.sqrt(3.)/2)*radius],
                [x-radius,y],
                [x-radius/2,y-(np.sqrt(3.)/2)*radius],
                [x+radius/2,y-(np.sqrt(3.)/2)*radius],
            ])

            if fill:
                kwargs=dict(mode="none", fill="toself", fillcolor="#ebebeb")
            else:
                kwargs=dict(mode="lines", fill=None, line_color="black", line_width=0.5, opacity=1)
                vertices = np.append(vertices, vertices[0:1, :], axis=0)

            self.fig.add_trace(go.Scatter(
                x = vertices[:, 0],
                y=vertices[:, 1],
                hoverinfo="skip", showlegend=False,
                **kwargs
            ))
        return self
    
    def addBarycenter(self, x, y):
        self.fig.add_trace(go.Scatter(
            mode="markers",
            name="Seed trackster barycenter",
            x=[x], y = [y],
            marker=dict(
                color="black",
                size=self.sizeSettings.get("impact.size", 8),
                symbol="x"
            ),
            hoverinfo='skip',
        ))
        return self
    
    def addMustache(self):
        # does not work
        seed_barycenter = vector.obj(x=self.event.leading_trackster.barycenter_x, y=self.event.leading_trackster.barycenter_y, z=self.event.leading_trackster.barycenter_z)
        dEta, dPhi = 0.1, 0.5
        z = self.event.leading_trackster.barycenter_z
        def xyFromEtaPhi(eta, phi):
            r_sintheta = z * 2* math.exp(eta) / (math.exp(2*eta) - 1)
            return r_sintheta * math.cos(phi), r_sintheta * math.sin(phi)
        def shiftEtaPhi(deta, dphi):
            return xyFromEtaPhi(seed_barycenter.eta+deta, seed_barycenter.phi+dphi)
        # corners = [
        #     shiftEtaPhi(dEta, dPhi),
        #     shiftEtaPhi(dEta,- dPhi),
        #     shiftEtaPhi(-dEta, -dPhi),
        #     shiftEtaPhi(-dEta, dPhi),
        # ]
        corners = []
        for d in np.linspace(dPhi, -dPhi, 300):
            corners.append(shiftEtaPhi(dEta, d))
        for d in np.linspace(dEta, -dEta, 50):
            corners.append(shiftEtaPhi(d, -dPhi))
        for d in np.linspace(-dPhi, dPhi, 300):
            corners.append(shiftEtaPhi(-dEta, d))
        for d in np.linspace(-dEta, -dEta, 50):
            corners.append(shiftEtaPhi(d, dPhi))
        #print(xyFromEtaPhi(seed_barycenter.eta, seed_barycenter.phi))
        # corners = [
        #     vector.obj(eta=seed_barycenter.eta+dEta, phi=seed_barycenter.phi+dPhi, rho=seed_barycenter.z),
        #     vector.obj(eta=seed_barycenter.eta-dEta, phi=seed_barycenter.phi-dPhi, rho=seed_barycenter.z),
        #     vector.obj(eta=seed_barycenter.eta+dEta, phi=seed_barycenter.phi-dPhi, rho=seed_barycenter.z),
        #     vector.obj(eta=seed_barycenter.eta-dEta, phi=seed_barycenter.phi+dPhi, rho=seed_barycenter.z),
        # ]
        # def makePoint(dEta, dPhi):
        #     return (seed_barycenter.z / math.tan(seed_barycenter.phi + dPhi), seed_barycenter.z * math.tan(2*math.atan(math.exp(-seed_barycenter.eta + dEta))))

        # corners = [makePoint(dEta, dPhi), makePoint(-dEta, -dPhi), makePoint(dEta, -dPhi), makePoint(-dEta, dPhi)]
        self.fig.add_trace(go.Scatter(
            x=[corner[0] for corner in corners],
            y=[corner[1] for corner in corners],
            # x = [corner.x for corner in corners],
            # y = [corner.y for corner in corners],
            hoverinfo="skip", showlegend=False,
            mode="lines",
            fill="none",
            line_color="black", line_width=3, line_dash="10px"
        ))
        #print(seed_barycenter)
        #print(corners)
        # print(list(zip([corner.x for corner in corners],
        #     [corner.y for corner in corners],)))
        return self
    
    def addSupercluster(self):
        eta_phi_sc = [(self.event.ts_df.loc[ts_id].barycenter_eta, self.event.ts_df.loc[ts_id].barycenter_phi) for ts_id in self.event.leading_supercluster][1:]
        z = self.event.leading_trackster.barycenter_z
        def xyFromEtaPhi(eta, phi):
            r_sintheta = z * 2* math.exp(eta) / (math.exp(2*eta) - 1)
            return r_sintheta * math.cos(phi), r_sintheta * math.sin(phi)
        self.fig.add_trace(go.Scatter(
            x=[xyFromEtaPhi(eta, phi)[0] for eta, phi in eta_phi_sc],
            y=[xyFromEtaPhi(eta, phi)[1] for eta, phi in eta_phi_sc],
            mode="markers",
            marker=dict(
                color="red",
                size=self.sizeSettings.get("impact.size", 8),
                symbol="x"
            ),
            hoverinfo='skip',
        ))
        return self
        

In [2]:
evt = LoadedEvent("/workspaces/repo/CMSSW_14_1_X_2024-08-26-2300/src/24813_eta2p3_phi_E300/histo_PU.root", 3, positiveEndcap=False)
evt.cmsswEventNb

3

In [15]:
layerNb = 10
size=1000
margin = 0
sizeSettings={"rechits.min" : 3, "rechits.max" : 40,#80,
               "rechits.opacity" : 1, "rechits.chain.minLineWidth" : 3, "rechits.chain.cumulativeEnergyFactor" : 0.0001, "rechits.chain.arrowSize" : 20,
            "clue.min":30, "clue.max" : 120, "clue.opacity":1,
            "impact.size":16}

override_colors = {evt.leading_trackster.name:"#00FE35", 694.:"#FD3216", 489.:"#00B5F7", 
    math.nan : "#222A2A"}

color_cycle_light = itertools.cycle(set(px.colors.qualitative.Light24).difference(override_colors.values()))
ts_color_light = {ts_id : next(color_cycle_light) for ts_id in evt.ts_df[evt.ts_df.tsCLUE3D_recoToSim_CP_score < 0.3]}
ts_color_light.update(override_colors)

color_cycle = itertools.cycle(set(px.colors.qualitative.Dark24).difference(override_colors.values()))
ts_color_map = {ts_id : next(color_cycle) for ts_id in evt.ts_df[((evt.ts_df.barycenter_x-evt.leading_trackster.barycenter_x)**2 + (evt.ts_df.barycenter_y-evt.leading_trackster.barycenter_y)**2) < 30**2].index}
ts_color_map.update(override_colors)

widthInDataCoords = 23
def makeFig(layerNb):
    z = evt.leading_trackster.barycenter_z
    def xyFromEtaPhi(eta, phi):
        r_sintheta = z * 2* math.exp(eta) / (math.exp(2*eta) - 1)
        return r_sintheta * math.cos(phi), r_sintheta * math.sin(phi)
    center_x, center_y = xyFromEtaPhi(evt.leading_trackster.barycenter_eta, evt.leading_trackster.barycenter_phi)
    
    fig = (LayerVisualization(evt, layerNb=layerNb, trackster_color_map=ts_color_map, sizeSettings=sizeSettings, center_x=center_x, center_y=center_y)
        #.addHexagonSensors(fill=False)
        .addMustache()
        .addRechits(energyForScale=evt.rechits_df[(evt.rechits_df.rechit_z * (1 if evt.positiveEndcap else -1) > 0) & (evt.rechits_df.rechit_layer < 26)].rechit_energy)
        .addBarycenter(center_x, center_y)
        #.addSupercluster()
    ).fig
    y_shift_factor=0.47
    fig.update_layout(xaxis_visible=False, yaxis_visible=False, title=None, showlegend=False, height=size, width=size*math.sqrt(2),
                    xaxis_showgrid=False, yaxis_showgrid=False, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)',
                    #xaxis_range=[-2.5, -2.5+widthInDataCoords], yaxis_range=[-7.5, -7.5+widthInDataCoords],
                    xaxis_range=[center_x-widthInDataCoords/2, center_x+widthInDataCoords/2], yaxis_range=[center_y-widthInDataCoords*y_shift_factor, center_y+widthInDataCoords*(1-y_shift_factor)],
                    margin=dict(l=margin, r=margin, t=margin, b=margin, pad=0))
    #fig.write_html("fig-test.html")
    dpi = 300
    width_a4_inches, height_a4_inches = 11.7, 8.3 
    fig.write_image(f"electron-v1/layer-{layerNb}.pdf", height=size, width=size*math.sqrt(2),
                   # width=width_a4_inches*dpi, height=height_a4_inches*dpi, scale=(height_a4_inches*dpi)/size
            )
    return fig
print(evt.leading_supercluster, [(evt.ts_df.loc[ts_id].barycenter_eta, evt.ts_df.loc[ts_id].barycenter_phi, evt.ts_df.loc[ts_id].tsCLUE3D_recoToSim_CP_sharedE*100/evt.ts_df.loc[ts_id].raw_energy) for ts_id in evt.leading_supercluster])
makeFig(4)

[696, 691] [(-2.292574644088745, -1.5791258811950684, 94.31808552231331), (-2.350992202758789, -1.3798915147781372, 0.9824593799305696)]


In [17]:
for layer in range(1, 26, 1):
    makeFig(layer)

In [ ]:
fig = makeFig(10)
z = evt.leading_trackster.barycenter_z
widthInDataCoordsNarrow = 10
def xyFromEtaPhi(eta, phi):
    r_sintheta = z * 2* math.exp(eta) / (math.exp(2*eta) - 1)
    return r_sintheta * math.cos(phi), r_sintheta * math.sin(phi)
center_x, center_y = xyFromEtaPhi(evt.leading_trackster.barycenter_eta, evt.leading_trackster.barycenter_phi)
center_x += 2
fig.update_layout(xaxis_range=[center_x-widthInDataCoordsNarrow/2, center_x+widthInDataCoordsNarrow/2], yaxis_range=[center_y-widthInDataCoordsNarrow*0.45, center_y+widthInDataCoordsNarrow*0.55],
                  )

In [ ]:
sensors = ak.to_dataframe(uproot.open("/workspaces/repo/CMSSW_14_1_X_2024-08-26-2300/src/24813_highEta_phiPi2/histo.root:ticlDumper/rechits").arrays(filter_name=["rechit_x", "rechit_y", "rechit_z", "rechit_layer"], library="ak"))
sensors = sensors.reset_index(drop=True).drop_duplicates().sort_values(["rechit_layer", "rechit_x", "rechit_y"])
sensors

,rechit_x,rechit_y,rechit_z,rechit_layer
1037409,-89.301758,-33.835197,322.154999,1
248131,-88.255257,4.229400,322.154999,1
823247,-87.557587,25.980598,-322.154999,1
320200,-86.162247,3.021000,322.154999,1
497995,-84.069237,-25.980598,322.154999,1
...,...,...,...,...
213899,88.255257,12.688199,-361.132996,26
248234,88.604088,-27.793198,361.132996,26
105863,91.743607,7.854599,361.132996,26
213898,96.627296,16.313398,-361.132996,26


## Poster legend

In [ ]:
fig = go.Figure(layout=dict(xaxis_visible=False, yaxis_visible=False, xaxis_showgrid=False, yaxis_showgrid=False, 
        plot_bgcolor='rgba(0,0,0,0)'),
    data=[
        # Rechits noPU mainTS
        # go.Scatter(
        #     mode="markers", x=[0], y=[0],
        #     marker=dict(
        #         symbol="circle",
        #         color="#2CA02C",
        #         size=200,
        #         opacity=1.,
        #     ),
        # ),
        go.Scatter(
            mode="markers", x=[1], y=[0],
            marker=dict(
                symbol="circle",
                color="rgba(0, 0, 0, 0)",
                size=200,
                opacity=1.,
                line=dict(
                    color=px.colors.qualitative.Dark24[0],
                    width=25,
                )
            ),
            opacity=1,
        ),
        # go.Scatter(
        #     mode="markers",
        #     x=[2], y = [0],
        #     marker=dict(
        #         color="black",
        #         size=200,
        #         symbol="x"
        #     ),
        # ),

    ])
fig.write_image("legend_rechits_PU.pdf")
fig

In [ ]:
fig = go.Figure(layout=dict(xaxis_visible=False, yaxis_visible=False, xaxis_showgrid=False, yaxis_showgrid=False, 
        plot_bgcolor='rgba(0,0,0,0)'),
    data=[
        # Rechits noPU mainTS
go.Scatter(
            x=[3, 5],
            y=[0, 0],
            # x = [corner.x for corner in corners],
            # y = [corner.y for corner in corners],
            hoverinfo="skip", showlegend=False,
            mode="lines",
            fill="none",
            line_color="black", line_width=30, line_dash="100px"
        )

    ])
fig.write_image("legend_box.pdf")
fig
